In [74]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import os

In [75]:
IDIR = os.path.abspath('../data')

print('loading newpriors')
newpriors = pd.read_csv(os.path.join(IDIR,'newpriors.csv'), dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8,
            'user_id' : np.int32,
            'eval_set' : 'category',
            'order_number': np.int16,
            'order_dow': np.int8,
            'order_hour_of_day': np.int8,
            'days_since_prior_order': np.float32})

print('loading products')
products = pd.read_csv(os.path.join(IDIR,'products.csv'), dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8},
        usecols=['product_id', 'aisle_id', 'department_id'])

print('loading userXproduct')
userXproduct = pd.read_csv(os.path.join(IDIR,'userXproduct.csv'), dtype={
            'user_id': np.int32,
            'product_id': np.int32,
            'UP_num_item': np.int16,
            'UP_first_order': np.int16,
            'UP_last_order' : np.int16,
            'user_num_orders' :np.int16,
            'UP_sum_add_cart' : np.int16,
            'UP_frequency': np.float16,
            'UP_average_add_cart': np.float16})

loading newpriors
loading products
loading userXproduct


In [76]:
print('computing product f')
prods = pd.DataFrame()
prods['orders'] = newpriors.groupby('product_id').size().astype(np.int32)
prods['reorders'] = newpriors.groupby('product_id')['reordered'].sum().astype(np.float32)
#prods['reorder_rate'] = (prods.reorders / prods.orders).astype(np.float32)
newproducts = products.join(prods, on='product_id')
newproducts.set_index('product_id', drop=False, inplace=True)
del prods

computing product f


# reorder rate of each product calcuation

In [77]:
#주석 추가 필요

In [78]:
Sum_of_user_orders_each_product = userXproduct.groupby('product_id').user_num_orders.sum()
#각 물품을 산 모든 유저들이 행한 주문의 총합

In [79]:
Sum_of_first_orders_each_product = userXproduct.groupby('product_id').UP_first_order.sum()
#각 물품에 대한 첫 번째 order number의 총 합(모든 유저들에 대해서)

In [80]:
num_of_orders_each_product = userXproduct.groupby('product_id').UP_num_item.sum()
#각 물품이 주문된 총 횟수(모든 유저들에 대해서)

In [81]:
num_of_users_each_product = userXproduct.groupby('product_id')['user_id'].size()
#각 물품을 구매한 유저들의 명수 

In [82]:
newproducts['reorder_rate'] = (num_of_orders_each_product - num_of_users_each_product)/(Sum_of_user_orders_each_product-Sum_of_first_orders_each_product)

In [85]:
products.head()

,product_id,aisle_id,department_id
0,1,61,19
1,2,104,13
2,3,94,7
3,4,38,1
4,5,5,13


In [83]:
newproducts.head()

,product_id,aisle_id,department_id,orders,reorders,reorder_rate
product_id,,,,,,
1,1,61,19,1852.0,1136.0,0.113714
2,2,104,13,90.0,12.0,0.011483
3,3,94,7,277.0,203.0,0.123030
4,4,38,1,329.0,147.0,0.087344
5,5,5,13,15.0,9.0,0.134328


In [84]:
products.to_csv(os.path.join(IDIR, 'newproducts.csv'), index=False)